<a href="https://colab.research.google.com/github/vitroid/PythonTutorials/blob/2020m0/2%20Advanced/024%E3%83%95%E3%82%A1%E3%82%A4%E3%83%AB%E5%85%A5%E5%87%BA%E5%8A%9B.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 複雑なファイルの入出力
研究でPythonを使う状況を考えると、何はともあれ大きなデータをファイルから読みこんで、処理して、ファイルに出力するという使い方がほとんどになるだろう。あらかじめPythonで読みこむつもりで書かれたデータばかりではないので、読み込むのにも工夫が必要になる。また、Pythonの出力を他のソフトウェアで利用する場合には、そのソフトウェアにあわせたデータ出力が求められる。

## ファイル入力

Google Colabでファイルを読み込む扱う場合、そのファイルがどこにあるかで、扱い方が違ってくる。とりあえず、ありそうなケースとしては、

1. ローカル(自分の端末上)にファイルがある。
1. Google Drive上にファイルがある。
1. インターネット上にファイルがある。
1. Colabファイル。

どの場合でも、できれば通常のローカルファイルと同じようにアクセスしたい。(ファイル全体をメモリに読みこむのではなく、必要に応じて必要量ずつ読みこむ)



### データの羅列を読む

In [0]:
#座標の羅列を読みこむ
file = open("data1.txt")
coord = []
for line in file:           #ファイルのそれぞれの行ごとに処理をする。
    columns = line.split()  #空白やタブで行をカラムに分割する。
    print("columns=",columns)
    x, y, z = float(columns[0]),float(columns[1]),float(columns[2])
    print("x,y,z=",x,y,z)
    coord.append((x,y,z))
    print("coord:",coord)
print(coord)
print(coord[1][1])

### もうちょっとシンプルに

In [0]:
#座標の羅列を読みこむ
file = open("data1.txt")
coord = []
for line in file:           #ファイルのそれぞれの行ごとに処理をする。
    values = [float(x) for x in line.split()]  #分割して実数化する
    print(line,values)
    coord.append(values)
print(coord)
values = [i for i in range(-5,10,2)]
print(values)

### 練習問題1
リスト[-5,-4,-3,-2,......,+4,+5]を作るPythonプログラムを1行で書いてみて下さい。

In [0]:
!! fortran90
integer i,a[11]
do i=-5,5
  a[i+6] = i
enddo

//c言語
int a[11]
for(int i=-5;i<=5;i++){
    a[i+5] = i;
}

### 練習問題2
リスト[-0.9,-0.8,-0.7,......,+0.9,+1.0]を作るPythonプログラムを1行で書いてみて下さい。

In [0]:
[i for i in range(-9,11)]

### 最初の3行は読みとばす

In [0]:
file = open("data2.txt")
coord = []
count = 0
for line in file:           #ファイルのそれぞれの行ごとに処理をする。
    count += 1
    if 3 < count:
        values = [float(x) for x in line.split()]  #分割して実数化する
        coord.append(values)
    else:
        print("Skipped line:",line,end="")
print(coord)

あるいは

In [0]:
file = open("data2.txt")
for i in range(3):
    file.readline()

coord = []
for line in file:           #ファイルのそれぞれの行ごとに処理をする。
    values = [float(x) for x in line.split()]  #分割して実数化する
    coord.append(values)
print(coord)

### 行のはじめに"@n"と書いてあり、そのあとのn行が座標データ

In [0]:
file = open("data3.txt")
while True:                 #無限ループ
    line = file.readline()
    if len(line) == 0:
        break
    if line[0] == "@":
        coord = []
        n = int(line[1:])
        for i in range(n):
            line = file.readline()
            values = [float(x) for x in line.split()]  #分割して実数化する
            coord.append(values)
        break               #whileループを抜ける
print(coord)

### \#以降はコメントとして無視する

In [0]:
# "#"よりもあとを除去する
def elim_comment(s):
    pos = s.find("#")
    if 0 <= pos:
        return s[0:pos]
    return s
    

file = open("data4.txt")
while True:                 #無限ループ
    line = file.readline()
    if line[0] == "@":
        coord = []
        n = int(line[1:])
        for i in range(n):
            line = elim_comment(file.readline())
            values = [float(x) for x in line.split()]  #分割して実数化する
            coord.append(values)
        break               #whileループを抜ける

print(coord)

### 行ごとではなく、列ごとに読みこむ

In [0]:
#座標の羅列を読みこむ
file = open("data1.txt")
columns = [[] for i in range(3)]
for line in file:           #ファイルのそれぞれの行ごとに処理をする。
    values = [float(x) for x in line.split()]  #分割して実数化する
    for i in range(3):
        columns[i].append(values[i])
print(columns)

### 行ごとに読みこんでから、転置する

In [0]:
def transpose(matrix, default=None):
    #列数の最大値を調べる。
    maxc = 0
    for row in matrix:
        if maxc < len(row):
            maxc = len(row)
    #コラムの数は列数の最大値
    columns = [[] for i in range(maxc)]
    for row in matrix:
        for i in range(maxc):
            if i < len(row):
                columns[i].append(row[i])
            else:
                #データが足りない部分はdefault値を入れる
                columns[i].append(default)
    return columns
    
file = open("data1.txt")
coord = []
for line in file:           #ファイルのそれぞれの行ごとに処理をする。
    values = [float(x) for x in line.split()]  #分割して実数化する
    coord.append(values)
print(coord)
columns = transpose(coord)
print(columns)
data = ["list","of","strings"]
print(data)
print(transpose(data,default=""))

In [0]:
from math import *
x = input("value?")
x = eval(x)
print(x)

In [0]:
eval("[i for i in range(10)]")

### 数値と文字が混在する行をパーズする。
try: except:構文を使うと、エラーが起こった場合にもプログラムを停止せず、その場で対応できます。この機能を使い、読みこんだ文字が数字か文字かをその場で判断します。

* 構文

try:
    エラーがおきるかもしれない処理
except エラータイプ:
    エラーがおこった場合の代替処理

エラータイプには、ValueErrorやTypeErrorなどがあります。実際のエラーメッセージを見れば、何を書けばいいかはわかると思います。

以下では、一行をカラムに分ける関数parse_properly()と、文字列を「てきとうな」値に変換する関数eval_properly()を準備しています。後者では、最初に整数だと決め打ちして型変換し、失敗したら実数だとみなし、それでも失敗したら文字列だとみなしています。複素数complex("j"を含む数値)も扱えるようにするにはどこを改造したらいいでしょう。

In [0]:
x = input("value?")
try:
    print(int(x))
except ValueError:
    print(x)

In [0]:
#Assume text does not contain spaces
def eval_properly(value):
    try:
        return int(value)
    except ValueError:
        try:
            return float(value)
        except ValueError:
            return value


def parse_properly(line,separator=None):
    columns = line.split(separator)
    newc = []
    for c in columns:
        newc.append(eval_properly(c))
    return newc


def parse_properly_in_short(line,separator=None):
    return [eval_properly(c) for c in line.split(separator)]

print(type(eval_properly("1 2 3")))
print(type(eval_properly("1.23")))
print(type(eval_properly("1.2+3j")))
print(type(eval_properly("123")))
print(parse_properly_in_short(input("test:"),separator=","))

## 宿題
data5.mdvwから、原子の座標と思われる部分だけを読みこんで、座標のリストを作って下さい。

それができたら、原子の名前も読みこみ、原子の種類ごとに別のリストに入れるようにしてみましょう。

### 蛇足1

Python 2では漢字混じりの文字列はアルファベットのみの文字列と区別する必要がありましたが、Python3ではもう文字列をunicodeとして使うのが標準になったようです。そればかりでなく、変数名や関数名にもunicodeが使えます。

上のプログラムは、日本語で次のように書いても問題ありません。

In [0]:
def 適当に評価(値):
    try:
        return int(値)
    except ValueError:
        try:
            return float(値)
        except ValueError:
            return 値


def 適当に行処理(line,区切り=None):
    カラム = line.split(区切り)
    新カラム = []
    for 値 in カラム:
        新カラム.append(適当に評価(値))
    return 新カラム


def 適当に行処理短(文字列,区切り=None):
    return [適当に評価(値) for 値 in 文字列.split(区切り)]

print(type(適当に評価("1 2 3")))
print(type(適当に評価("1.23")))
print(type(適当に評価("1.2+3j")))
print(type(適当に評価("123")))
print(適当に行処理短(input("適当な文字列:"),区切り=","))

### 蛇足2

この次に扱うnumpyでは、数値をまとめて読みこむ便利な関数がつかえます。